# Download fake news data

This notebook downloads the fake news data from kaggle and does some first analysis.

In [ ]:
%%capture

# download required packages
!pip install kaggle
!pip install zipfile

In [ ]:
# import libraries
from kaggle.api.kaggle_api_extended import KaggleApi
from zipfile import ZipFile
import pandas as pd
import os

In [ ]:
# user parameters
data_path = 'data'

In [ ]:
def download_kaggle_data(data_path, data_set, extract_zip=True):
    
    print('Download {} from kaggle.com...'.format(data_set), end='')
    api = KaggleApi()
    api.authenticate()
    api.dataset_download_file(
        dataset='clmentbisaillon/fake-and-real-news-dataset',
        file_name=data_set,
        path=data_path,
        force=True
    )

    file_path = '{}/{}.zip'.format(data_path, data_set)
    
    if extract_zip:
        zf = ZipFile(file_path)
        zf.extractall(data_path)
        zf.close()
        !rm $file_path

    print('done')

In [ ]:
# download fake news data
download_kaggle_data(data_path, 'Fake.csv')
download_kaggle_data(data_path, 'True.csv')

In [ ]:
def read_kaggle_data(data_path, data_set):
    
    file_path = '{}/{}'.format(data_path, data_set)
    if os.path.exists(file_path):
        
        print('Import {}...'.format(data_set), end='')
    
        file_path = '{}/{}'.format(data_path, data_set)
        imported_data = pd.read_csv(file_path)
        !rm $file_path
    
        print('done')
    
    else:
        
        empty_dict = {'dummy_col_1': [3, 2, 1, 0], 'dummy_col_2': ['a', 'b', 'c', 'd']} # just some test data
        imported_data = pd.DataFrame.from_dict(empty_dict)
        print('{} does not exist. Please download again kaggle data.'.format(file_path))
    
    return imported_data

In [ ]:
# import data
fake = read_kaggle_data(data_path, 'Fake.csv')
true = read_kaggle_data(data_path, 'True.csv')

In [ ]:
# add class 
fake['class'] = 1 # 1 = fake
true['class'] = 0 # 0 = true

In [ ]:
fake.head()

In [ ]:
fake[['title', 'text', 'subject', 'date']].describe()

In [ ]:
true.head()

In [ ]:
true[['title', 'text', 'subject', 'date']].describe()

We can see here that we have less unique values for title and text as counted values. This means several times the same titles and texts occur. I will remove not unique features once the text is processed further. 

In [ ]:
# merge data together and add classification
original_document = pd.concat([true, fake])
true = fake = None # to save memory

In [ ]:
original_document.head()

In [ ]:
original_document[['title', 'text', 'subject', 'date']].describe()

In [ ]:
def detailed_investigation(index, df, number_strings):
    doc_class = df['class'].iloc[index]
    title = df['title'].iloc[index]
    text = df['text'].iloc[index]
    text = text[:number_strings]
    print('Class {} - {}\n{}\n'.format(doc_class, title, text))

In [ ]:
detailed_investigation(11010, original_document, 150)

In [ ]:
select_from = 40000

In [ ]:
for i in range(select_from, select_from + 5000):
    detailed_investigation(i, original_document, 300)